In [1]:
from datasets import load_dataset
import numpy as np
import pandas as pd
import polars as pl
from sklearn.metrics import average_precision_score
from tqdm import tqdm

/accounts/projects/yss/gbenegas/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset_path = "gonzalobenegas/siraj_gwas_highpip"
V = load_dataset(dataset_path, split="test").to_pandas()
V

,chrom,pos,ref,alt,label
0,1,3080038,T,C,True
1,1,3774964,A,G,True
2,1,6616131,C,T,False
3,1,7665224,C,A,False
4,1,8407293,G,A,False
...,...,...,...,...,...
1778,22,47019717,G,T,False
1779,22,47990921,C,T,True
1780,22,50007172,T,C,False
1781,22,50190508,G,A,True


In [5]:
#features = "GPN-MSA_absLLR"
features = "GPN-MSA_InnerProducts"
df = pd.read_parquet(f"https://huggingface.co/datasets/{dataset_path}/resolve/main/features/{features}.parquet")
feature_cols = df.columns
V = pd.concat([V, df], axis=1)
V

,chrom,pos,ref,alt,label,score,embedding_0,embedding_1,embedding_2,embedding_3,...,embedding_758,embedding_759,embedding_760,embedding_761,embedding_762,embedding_763,embedding_764,embedding_765,embedding_766,embedding_767
0,1,3080038,T,C,True,0.265625,58.531685,61.196415,275.071045,341.320740,...,71.805069,178.218781,77.948822,99.718391,51.739311,67.689011,94.278885,195.173981,47.125771,151.364716
1,1,3774964,A,G,True,0.366699,90.261459,63.717766,245.317688,268.411591,...,78.819626,205.081055,65.767632,134.453003,47.582321,71.301094,77.618454,176.077698,49.061073,121.118881
2,1,6616131,C,T,False,0.084473,250.107437,89.565338,241.990463,302.441650,...,116.357117,390.233246,33.388409,154.918945,40.924004,47.359383,70.278290,130.302673,59.024097,79.366638
3,1,7665224,C,A,False,2.230469,79.299088,156.702026,321.146729,438.729614,...,92.475922,166.541046,85.150772,115.177994,63.518593,53.100800,131.995544,176.944244,62.428444,234.591156
4,1,8407293,G,A,False,1.397461,54.762707,76.711212,236.164383,268.850800,...,59.512550,183.969009,96.003418,121.622818,49.704277,64.489792,82.471321,266.826050,63.760647,154.062927
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1778,22,47019717,G,T,False,0.123779,77.499039,148.730087,433.772705,412.336426,...,148.611603,236.271362,76.711716,166.230820,42.370628,59.562469,63.274250,165.583176,63.191666,113.216721
1779,22,47990921,C,T,True,1.453125,49.160408,281.167053,468.288879,584.203613,...,59.851269,85.045303,54.344757,74.617432,80.003174,104.478195,40.364479,251.294632,103.200821,110.261353
1780,22,50007172,T,C,False,0.938965,90.186684,77.209351,275.260071,151.026443,...,67.989235,310.427032,46.227379,168.898666,80.758728,44.292622,16.692547,249.116318,69.971817,108.223160
1781,22,50190508,G,A,True,1.195312,104.036636,41.170593,190.614853,391.268311,...,95.924118,196.216095,71.016647,86.233978,28.755674,66.285851,103.441513,136.004608,25.243877,160.444504


In [6]:
# absLLR: 0.62

res = []
for c in tqdm(feature_cols):
    res.append([c, max(average_precision_score(V["label"], V[c]), average_precision_score(V["label"], -V[c]))])
res = pd.DataFrame(res, columns=["feature", "AUPRC"]).sort_values("AUPRC", ascending=False)
res

100%|█████████████████████████████████████████████████████████| 768/768 [00:02<00:00, 296.92it/s]


,feature,AUPRC
684,embedding_684,0.696329
112,embedding_112,0.691407
565,embedding_565,0.689875
14,embedding_14,0.681853
132,embedding_132,0.681131
...,...,...
195,embedding_195,0.477496
717,embedding_717,0.476910
392,embedding_392,0.474966
536,embedding_536,0.473449
